# Simulate the ground state of a Hydrogen molecule using Variational Quantum Eigensolver (VQE) on Azure Quantum.

![Hydrogen molecule](https://user-images.githubusercontent.com/4041805/166981145-c33b8d1a-24d1-4776-91ee-f514b0a5ab04.jpg)

In this notebook, you'll learn how to run VQE for a $H_{2}$ molecule using Qiskit on an Azure Quantum backend.

VQE is a variational algorithm for quantum chemistry that uses an optimization loop to minimize a cost function. The cost function is an energy evaluation $E = \left\langle\psi|H|\psi\right\rangle$ where $|\psi (\theta)\rangle$ is a parametric trial state that estimates the ground state of the molecule. For each evaluation, we modify the trial state until the energy reaches a minimum.

![VQE diagram](https://user-images.githubusercontent.com/4041805/166981008-023aba4c-26f8-498e-93ee-a1d9a39ddbcd.png)

For more information about running VQE using Qiskit, see: [Qiskit Textbook - VQE Molecules](https://qiskit.org/textbook/ch-applications/vqe-molecules.html#implementationnoisy).

To read more about the optimization method used in this example, see [Wikipedia - SPSA](https://en.wikipedia.org/wiki/Simultaneous_perturbation_stochastic_approximation).

## Define  problem

You will use an FCIDUMP file for an $H_2$ molecule as input to the VQE run. More information about the FCIDUMP file format is [here](https://www.sciencedirect.com/science/article/abs/pii/0010465589900337). 

You will first download the FCIDUMP file in the cell below.

In [ ]:
import requests

# TODO: Add link to FCIDump file
url = ''
r = requests.get(url, allow_redirects=True)

open('vqe_h2.fcidump', 'wb').write(r.content)

Then define how to generate the problem instance and the `QubitConverter` that would be used as inputs to the VQE run. 

Here you will use the `fcidump_to_problem` Qiskit function to generate the problem instance.

The [Jordan-Wigner transformation](https://en.wikipedia.org/wiki/Jordan%E2%80%93Wigner_transformation) is specified as the `QubitConverter`to use.


In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.formats import  fcidump_to_problem
from qiskit_nature.second_q.formats.fcidump import FCIDump
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

def get_problem():
    return fcidump_to_problem(FCIDump.from_file("vqe_h2.fcidump"))

def get_qubit_converter():
    return QubitConverter(JordanWignerMapper(), two_qubit_reduction=True, z2symmetry_reduction='auto')


## Exact result
First, let's compute the exact result that could be used to compare with the results of VQE later on. 

In [ ]:
from qiskit_nature.second_q.algorithms import NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

problem = get_problem()
qubit_converter = get_qubit_converter()
exact_solver = NumPyMinimumEigensolverFactory()
calc = GroundStateEigensolver(qubit_converter, exact_solver)
result = calc.solve(problem)


In [ ]:
print("Exact result:\n")
print(result.groundenergy)

## Run on Qiskit's Local AER Simulator without Noise

Here, you will simulate the VQE run locally using the noiseless Aer simulator. Here the maximum number of iterations of the optimizer is set to 100.

In [ ]:
# define Aer Estimator for noiseless statevector simulation
from qiskit.algorithms.optimizers import SPSA
from qiskit.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD

problem = get_problem()
qubit_converter = get_qubit_converter()

spsa_optimizer=SPSA(maxiter=100)

noiseless_estimator = AerEstimator(
    run_options={ "shots": 1000 }
)

vqe_solver = VQEUCCFactory(estimator=noiseless_estimator, ansatz=UCCSD(), optimizer=spsa_optimizer)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
result = calc.solve(problem)

In [ ]:
print("Local AER simulator result:\n")
print(result.groundenergy)

## Run on the IonQ Simulator through Azure Quantum
Now, you will run the same problem on the IonQ simulator through Azure Quantum. It is also possible to run it on other backends. Here the maximum number of iterations of the optimizer is set to 50.


In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider

# Connect to the Azure Quantum workspace via a Qiskit provider
provider = AzureQuantumProvider(
            resource_id = "",
            location = "")

In [ ]:
ionq_sim = provider.get_backend('ionq.simulator')
quantinuum_sim = provider.get_backend('quantinuum.sim.h1-1e')
rigetti_sim = provider.get_backend('rigetti.sim.qvm')

# Set the backend you want to use here.
backend_to_use = ionq_sim


You will now kickoff the VQE run on the selected backend through Azure Quantum. Since this will trigger several jobs, a `session` is used to group them together into a single logical entity.

**IMPORTANT !!**

1. This cell will take about 15 minutes to run on the IonQ simulator for max_iter=50. It generates around 300 jobs. The time can vary depending on the backend queue times. Running with max_iter=50 may not be sufficient for the results to converge. You could consider increasing the number of iterations to 100 to give a more accurate result, but please be aware of the increased running times for the cell.
1. You may lose results if you lose network connectivity while the cell is running. It may be better to download the notebook and run it locally for better reliability.
1. If you run this against a QPU harware backend instead of a simulator, you will likely incur a large cost or consume a large number of your alloted credits.
1. If the `calc.solve` fails, one or more jobs may have failed. If so, you can find the session under `Job management` in your workspace, click on it to open the Session view and then click on a Job that failed to find the reason for failure.



In [ ]:

from qiskit_nature.second_q.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit.primitives.backend_estimator import BackendEstimator

problem = get_problem()
qubit_converter = get_qubit_converter()

spsa_optimizer=SPSA(maxiter=50)

backend_estimator = BackendEstimator(backend=backend_to_use, options={ "shots": 1000 })
vqe_solver = VQEUCCFactory(estimator=backend_estimator, ansatz=UCCSD(), optimizer=spsa_optimizer)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)

# Wrap the call to solve in a session "with" block so that all jobs submitted by it are grouped together.
with backend_to_use.open_session(name="VQE H2") as session:
    result = calc.solve(problem)

In [ ]:
print("AzureQuantum " + backend_to_use.name() + " result:\n")

In this notebook, you've run VQE on an Azure Quantum backend to calculate the ground state of a $H_2$ molecule. Nice job! 👏🏽

As a next step, you can modify the sample to run your own molecule, or run it on QPU hardware.

